# 4 Modeling 

## 4.1 Contents<a id='4.1_Contents'></a>
* [4 Modeling](#4_Modeling)
  * [4.1 Contents](#4.1_Contents)
  * [4.2 Introduction](#4.2_Introduction)
  * [4.3 Imports](#4.3_Imports)
  * [4.4 Load The Data](#4.4_Load_The_Data)
    * [4.4.1 Explore The Data](#4.4.1_Explore_The_Data)
    * [4.4.2 Fixing Categorical Data](#4.4.2_Fixing_Data)
  * [4.5 Train/Test_Split](#4.5_Train/Test_Split)
  * [4.6 Initial Average Mode](#4.6_Average_Model)
  * [4.7 Metrics](#4.7_Metrics)
  * [4.8 Imputations](#4.8_Imputations)
  * [4.9 Dimensitionality Reduction](#4.9_Dimensionality)
      * [4.9.1 Manual Imputation](#4.9.1_Manual_Imputations)   
  * [4.10 Gradient Boosting Model](#4.10_Gradient_Boosting)
  * [4.11 Hyper-parameter Tuning Gradient Boost](#4.11_Hyper_parameter_Tuning)
  * [4.12 Best Model](#4.12_Best_Model)

## 4.2 Introduction<a id='4.2_Introduction'></a>
#### In this notebook we take our preproccessed data from our Preprocessing notebook and try different models to see which yield the best performance and tune our hyper-parameters to maximize best results. Quick disclaimer the data used in the prior notebooks have been superseded by a new dataframe, while the dataset is the same I went back and redid my final dataset which has more observation and a different set of features. I felt that while the world I did originally was fine, but I needed to test few parameters and different features so I will be using a new feature set but for the most part everything else is the same. I also implement a MICE imputation method rather than just imputation of mean values for each feature set

#### With the goal to make the best model which predicts house prices in the NYC market for capital fortune we have done a lot of data work cleaning and wrangling a very messy zillow dataset with various missing values and messy feature set. But after some tedious work we were able to establish a workable dataset to train our model. 

## 4.3 Imports<a id='4.3_Imports'></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import scipy.stats as stats
import matplotlib.ticker as tick
import sklearn.model_selection

from operator import itemgetter
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier)
import xgboost as xgb

import featuretools as ft
from sklearn import neighbors, datasets, preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve